## Starting a Time Series Project

**Author**: Thodoris Petropoulos

**Label**: Modeling Options

### Scope
The scope of this notebook is to provide instructions on how to initiate a DataRobot project for a Time Series problem using the R API. This notebook also covers calendars and feature settings for time series projects.

### Background
Time series modeling is the use of a machine learning model to predict future values of the target column based on previously observed values.

Examples:

- Demand Forecasting
- Staffing
- Sales Forecast

### Requirements

- Python version 3.7.3
-  DataRobot API version 2.19.0. 
Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com

#### Import Libraries

In [1]:
import datarobot as dr
import pandas as pd
import numpy as np

#### Dataset
For this tutorial, we are going to generate some simple time series data.

In [2]:
intervals = 10 * 24 * 60 #10 Days, 24 hours, 60 minutes for each our
time_index = pd.date_range('2020-01-01', periods=intervals, freq='T')
data = np.random.randn(intervals)

df = pd.DataFrame(pd.Series(data=data, index=time_index)) #Create pandas dataframe with data
df.reset_index(inplace=True) #Reset index
df.columns = ['date','target'] #Rename columns
df.head()

,date,target
0,2020-01-01 00:00:00,2.210569
1,2020-01-01 00:01:00,0.257881
2,2020-01-01 00:02:00,-0.397363
3,2020-01-01 00:03:00,0.242060
4,2020-01-01 00:04:00,0.160361


#### Connect to DataRobot
Connect to DataRobot using your credentials and your endpoint. Change input below accordingly.

In [ ]:
dr.Client(token='YOUR_API_KEY', 
          endpoint='YOUR_DATAROBOT_HOSTNAME')

#### Initiate Time Series Project
To set up a time series project, use the new time series specific parameters found in <code>dr.DatetimePartitioningSpecification </code>:

* use_time_series - set this to True to enable time series for the project.
* default_to_known_in_advance - set this to True to default to treating all features as known in advance features. Otherwise they will not be handled as known in advance features. See the prediction documentation for more information.
* feature_derivation_window_start - the offset into the past to the start of the feature derivation window.
* feature_derivation_window_end - the offset into the past to the end of the feature derivation window.
* forecast_window_start - the offset into the future to the start of the forecast window.
* forecast_window_end - the offset into the future to the end of the forecast window.
* feature_settings - A list of settings. Can be used to set individual features to “known in advance”.
* treat_as_exponential - Used to specify whether to treat the data as an exponential trend, which will apply a log-transform. By default, set as “auto”, this can be inferred automatically. See possible values in TreatAsExponential.
* differencing_method - Used to specify a differencing method to apply if data is stationary. By default, set as “auto”, this can be inferred automatically. See possible values in DifferenicngMethod.
* periodicities - A list of periodicities of different timestamps, specified in a list of lists.
* windows_basis_units - The unit to use for feature derivation and forecast windows. Defaults to the inferred time step. If "ROW", will define the window with a number of rows.

In [ ]:
dr.DatetimePartitioningSpecification()

In [ ]:
#First create the project
project = dr.Project.create(project_name='MySingleTimeSeriesProject',
                        sourcedata= df)

#Define a DatetimePartitioningSpecification object
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    use_time_series= True)

# start the project, specifying the partitioning method
project.set_target(
    target='target',
    partitioning_method=partition
)
project.wait_for_autopilot() #Wait for autopilot to complete

#### Feature Derivation Window

The Feature Derivation window represents the rolling window that is used to derive time series features and lags, relative to the Forecast Point. It is defined in terms of <code>feature_derivation_window_start</code> and <code>feature_derivation_window_end</code> which are integer values representing datetime offsets in terms of the unit of time (e.g. hours or days).
The Feature Derivation Window start and end must be less than or equal to zero, indicating they are positioned before the forecast point. Additionally, the window must be specified as an integer multiple of the time step which defines the expected difference in time units between rows in the data.
Enough rows of historical data must be provided to cover the span of the effective Feature Derivation Window (which may be longer than the project’s Feature Derivation Window depending on the differencing settings chosen). The effective Feature Derivation Window of any model can be checked via the <code>Project.effective_feature_derivation_window_start</code> and <code>effective_feature_derivation_window_end</code> attributes of a datetime model. See <code>Project.get_datetime_models</code>.
The window is closed, meaning the edges are considered to be inside the window.
This information is added to your <code>DatetimePartitioningSpecification</code> call like so:


In [ ]:
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    feature_derivation_window_start = -24,
    feature_derivation_window_end = -12,
    use_time_series= True)

#### Forecast Window
The Forecast Window represents the rolling window of future values to predict, relative to the Forecast Point. It is defined in terms of the <code>forecast_window_start</code> and <code>forecast_window_end</code>, which are positive integer values indicating datetime offsets in terms of the time unit (e.g. hours or days).
The Forecast Window start and end must be positive integers, indicating they are positioned after the forecast point. Additionally, the window must be specified as an integer multiple of the time step which defines the expected difference in time units between rows in the data.
The window is closed, meaning the edges are considered to be inside the window.
This information is added to your <code>DatetimePartitioningSpecification</code> call like so:

In [ ]:
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    forecast_window_start = 1,
    forecast_window_end = 7,
    use_time_series= True)

#### Feature Settings - Enable/Disable known in advance

When setting up a time series project, input features could be identified as known in advance features. These features are not used to generate lags, and are expected to be known for the rows in the forecast window at predict time (e.g. “how much money will have been spent on marketing”, “is this a holiday”).
To start a time series project, use <code>DatetimePartitioningSpecification</code> and specify the <code>feature_settings</code>. (Note that this is for illustrative purposes only - this project will not actually build because the 10 data points are smaller than the 100 datapoint minimum required.)

In [ ]:
settings = [dr.FeatureSettings('known_in_advance_feature', known_in_advance=True)]
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    use_time_series= True,
    feature_settings = settings)

#### Feature Settings - Enable/Disable feature derivation

DataRobot does a lot of good work to automatically derive features that may be useful (e.g., lags). However, from time to time, it may be useful to disable DataRobot’s automatic feature engineering for a particular feature (e.g., so you can derive lags yourself manually). To do this, we can use the <code>feature_settings</code> to turn off derived features for a particular base feature:

In [ ]:
settings = [dr.FeatureSettings('do_not_derive_feature', do_not_derive=True)]
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    use_time_series= True,
    feature_settings = settings)

#### Multiseries
The API also supports <strong>multiseries</strong>, or data with multiple time series delineated by multiseries ID columns. To create this, create a project, then create a datetime partition specification that specifies the <code>datetime_partition_column</code> (the column with your date in it) and the <code>multiseries_id_columns</code> (a list of columns specifying the ids that delineate the multiseries).

In [ ]:
partition = dr.DatetimePartitioningSpecification(
    datetime_partition_column = 'date',
    use_time_series= True,
    multiseries_id_columns=['multiseries_id'])

#### Prediction Intervals
For each model, prediction intervals estimate the range of values DataRobot expects actual values of the target to fall within. They are similar to a confidence interval of a prediction, but are based on the residual errors measured during the backtesting for the selected model.
Note that because calculation depends on the backtesting values, prediction intervals are not available for predictions on models that have not had all backtests completed. Additionally, prediction intervals are not available when the number of points per forecast distance is less than 10, due to insufficient data.
In a prediction request, users can specify a prediction intervals size, which specifies the desired probability of actual values falling within the interval range. Larger values are less precise, but more conservative. For example, specifying a size of 80 will result in a lower bound of 10% and an upper bound of 90%. More generally, for a specific <code>prediction_intervals_size</code>, the upper and lower bounds will be calculated as follows:
<ul>
<li>predictionIntervalUpperBound = 50% + (<code>prediction_intervals_size</code> / 2)</li>
<li>predictionIntervalLowerBound = 50% - (<code>prediction_intervals_size</code> / 2)</li>
</ul>
To view prediction intervals data for a prediction, the prediction needs to have been created using <code>request_predictions</code> and specifying <code>include_prediction_intervals = True</code>. The size for the prediction interval can be specified with the <code>prediction_intervals_size</code> parameter for the same function, and will default to 80 if left unspecified. Specifying these fields will result in prediction interval bounds being included in the retrieved prediction data for that request. See <code>request_predictions</code> for more details.

#### Calendars
A calendar is an external file that lists events for a time series, such as holidays. For example, we might consider this calendar:

In [ ]:
calendar_raw = """01/01/2019;New Years,04/01/2019;National Spaghetti Day,29/11/2019;Black Friday"""
calendar = pd.DataFrame([x.split(';') for x in calendar_raw.split(',')],columns=['date','event'])
calendar.head()

To create a calendar file from a csv file:

In [ ]:
calendar.to_csv('calendar.csv',index=False) #Save Pandas dataframe as csv first
calendar = dr.CalendarFile.create('calendar.csv')

#### Making a Time Series Project using a Calendar
The main point of having calendars is not to admire them, but to use them for time series modeling! To do this, make a datetime partition like you usually would and pass the calendar using the calendar parameter.

In [ ]:
# specify the calendar_id in the partitioning specification
datetime_spec = dr.DatetimePartitioningSpecification(
    use_time_series=True,
    datetime_partition_column='date'
    calendar_id=calendar.id
)

#### Listing all available calendars
To list all of the available calendars use the dr.CalendarFile.list command

In [ ]:
dr.CalendarFile.list()